# LangGraph and LangSmith - Agentic RAG Powered by LangChain

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating our Tool Belt
  4. Creating Our State
  5. Creating and Compiling A Graph!

- 🤝 Breakout Room #2:
  1. Evaluating the LangGraph Application with LangSmith
  2. Adding Helpfulness Check and "Loop" Limits
  3. LangGraph for the "Patterns" of GenAI

# 🤝 Breakout Room #1

## Part 1: LangGraph - Building Cyclic Applications with LangChain

LangGraph is a tool that leverages LangChain Expression Language to build coordinated multi-actor and stateful applications that includes cyclic behaviour.

### Why Cycles?

In essence, we can think of a cycle in our graph as a more robust and customizable loop. It allows us to keep our application agent-forward while still giving the powerful functionality of traditional loops.

Due to the inclusion of cycles over loops, we can also compose rather complex flows through our graph in a much more readable and natural fashion. Effectively allowing us to recreate application flowcharts in code in an almost 1-to-1 fashion.

### Why LangGraph?

Beyond the agent-forward approach - we can easily compose and combine traditional "DAG" (directed acyclic graph) chains with powerful cyclic behaviour due to the tight integration with LCEL. This means it's a natural extension to LangChain's core offerings!

## Task 1:  Dependencies


## Task 2: Environment Variables

We'll want to set our OpenAI, Tavily, and LangSmith API keys along with our LangSmith environment variables.

In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [2]:
os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY")

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE8 - LangGraph - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

## Task 3: Creating our Tool Belt

As is usually the case, we'll want to equip our agent with a toolbelt to help answer questions and add external knowledge.

There's a tonne of tools in the [LangChain Community Repo](https://github.com/langchain-ai/langchain-community/tree/main/libs/community) but we'll stick to a couple just so we can observe the cyclic nature of LangGraph in action!

We'll leverage:

- [Tavily Search Results](https://github.com/langchain-ai/langchain-community/blob/main/libs/community/langchain_community/tools/tavily_search/tool.py)
- [Arxiv](https://github.com/langchain-ai/langchain-community/blob/main/libs/community/langchain_community/tools/arxiv/tool.py)

#### 🏗️ Activity #1:

Please add the tools to use into our toolbelt.

> NOTE: Each tool in our toolbelt should be a method.

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.arxiv.tool import ArxivQueryRun

tavily_tool = TavilySearchResults(max_results=5)

tool_belt = [
    tavily_tool,
    ArxivQueryRun(),
]

/var/folders/wj/phn3tgld2kv18rx1l1kkq57r0000gn/T/ipykernel_11887/1203815797.py:4: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_tool = TavilySearchResults(max_results=5)


### Model

Now we can set-up our model! We'll leverage the familiar OpenAI model suite for this example - but it's not *necessary* to use with LangGraph. LangGraph supports all models - though you might not find success with smaller models - as such, they recommend you stick with:

- OpenAI's GPT-3.5 and GPT-4
- Anthropic's Claude
- Google's Gemini

> NOTE: Because we're leveraging the OpenAI function calling API - we'll need to use OpenAI *for this specific example* (or any other service that exposes an OpenAI-style function calling API.

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

Now that we have our model set-up, let's "put on the tool belt", which is to say: We'll bind our LangChain formatted tools to the model in an OpenAI function calling format.

In [6]:
model = model.bind_tools(tool_belt)

#### ❓ Question #1:

How does the model determine which tool to use?
lets see Arxiv:

-------------------------------------------------------------------------------------------------------------------------
##### ✅ Answer:

* **Simple answer**: The model chooses tools by reading their _descriptions_ and using its reasoning abilities to match the user's question with the most appropriate tool.

Let's see [arxiv](https://python.langchain.com/api_reference/_modules/langchain_community/tools/arxiv/tool.html#ArxivQueryRun)
```python
        name: str = "arxiv"
        description: str = (                 #<---------------  this is the description
        "A wrapper around Arxiv.org "
        "Useful for when you need to answer questions about Physics, Mathematics, "
        "Computer Science, Quantitative Biology, Quantitative Finance, Statistics, "
        "Electrical Engineering, and Economics "
        "from scientific articles on arxiv.org. "
        "Input should be a search query."
```

-------------------------------------------------------------------------------------------------------------------------

## Task 4: Putting the State in Stateful

Earlier we used this phrasing:

`coordinated multi-actor and stateful applications`

So what does that "stateful" mean?

To put it simply - we want to have some kind of object which we can pass around our application that holds information about what the current situation (state) is. Since our system will be constructed of many parts moving in a coordinated fashion - we want to be able to ensure we have some commonly understood idea of that state.

LangGraph leverages a `StatefulGraph` which uses an `AgentState` object to pass information between the various nodes of the graph.

There are more options than what we'll see below - but this `AgentState` object is one that is stored in a `TypedDict` with the key `messages` and the value is a `Sequence` of `BaseMessages` that will be appended to whenever the state changes.

Let's think about a simple example to help understand exactly what this means (we'll simplify a great deal to try and clearly communicate what state is doing):

1. We initialize our state object:
  - `{"messages" : []}`
2. Our user submits a query to our application.
  - New State: `HumanMessage(#1)`
  - `{"messages" : [HumanMessage(#1)}`
3. We pass our state object to an Agent node which is able to read the current state. It will use the last `HumanMessage` as input. It gets some kind of output which it will add to the state.
  - New State: `AgentMessage(#1, additional_kwargs {"function_call" : "WebSearchTool"})`
  - `{"messages" : [HumanMessage(#1), AgentMessage(#1, ...)]}`
4. We pass our state object to a "conditional node" (more on this later) which reads the last state to determine if we need to use a tool - which it can determine properly because of our provided object!

In [7]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

## Task 5: It's Graphing Time!

Now that we have state, and we have tools, and we have an LLM - we can finally start making our graph!

Let's take a second to refresh ourselves about what a graph is in this context.

Graphs, also called networks in some circles, are a collection of connected objects.

The objects in question are typically called nodes, or vertices, and the connections are called edges.

Let's look at a simple graph.

![image](https://i.imgur.com/2NFLnIc.png)

Here, we're using the coloured circles to represent the nodes and the yellow lines to represent the edges. In this case, we're looking at a fully connected graph - where each node is connected by an edge to each other node.

If we were to think about nodes in the context of LangGraph - we would think of a function, or an LCEL runnable.

If we were to think about edges in the context of LangGraph - we might think of them as "paths to take" or "where to pass our state object next".

Let's create some nodes and expand on our diagram.

> NOTE: Due to the tight integration with LCEL - we can comfortably create our nodes in an async fashion!

In [11]:
# Step 1: Define what the nodes DO (the functions/logic)

from langgraph.prebuilt import ToolNode

# Node 1: Define the agent logic
def call_model(state):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages": [response]}

# Node 2: Define the tool logic
tool_node = ToolNode(tool_belt)

Now we have two total nodes. We have:

- `call_model` is a node that will...well...call the model
- `tool_node` is a node which can call a tool

Let's start adding nodes! We'll update our diagram along the way to keep track of what this looks like!


In [12]:
# Step 2: Add the nodes to the graph

from langgraph.graph import StateGraph, END

# Create an empty graph
uncompiled_graph = StateGraph(AgentState)

# Add Node 1 to the graph with name "agent"
uncompiled_graph.add_node("agent", call_model)

# Add Node 2 to the graph with name "action"
uncompiled_graph.add_node("action", tool_node)

Let's look at what we have so far:

![image](https://i.imgur.com/md7inqG.png)

Next, we'll add our entrypoint. All our entrypoint does is indicate which node is called first.

In [13]:
uncompiled_graph.set_entry_point("agent")

![image](https://i.imgur.com/wNixpJe.png)

Now we want to build a "conditional edge" which will use the output state of a node to determine which path to follow.

We can help conceptualize this by thinking of our conditional edge as a conditional in a flowchart!

Notice how our function simply checks if there is a "function_call" kwarg present.

Then we create an edge where the origin node is our agent node and our destination node is *either* the action node or the END (finish the graph).

It's important to highlight that the dictionary passed in as the third parameter (the mapping) should be created with the possible outputs of our conditional function in mind. In this case `should_continue` outputs either `"end"` or `"continue"` which are subsequently mapped to the action node or the END node.

In [14]:
def should_continue(state):
  last_message = state["messages"][-1]    # Look at the last message

  if last_message.tool_calls:             # Did the agent want to use a tool?
    return "action"                       # Yes → go to "action" node

  return END                              # No → we're done, end the graph

uncompiled_graph.add_conditional_edges(
    "agent",                              # FROM the "agent" node
    should_continue                       # USE this function to decide where to go
)

```bash
       START
         ↓
       [agent] → (should_continue checks state)
                     ↓                    ↓
              tool_calls exists?       No tool_calls?
                     ↓                    ↓
                 [action]                END ✓
```

```

Let's visualize what this looks like.

![image](https://i.imgur.com/8ZNwKI5.png)

Finally, we can add our last edge which will connect our action node to our agent node. This is because we *always* want our action node (which is used to call our tools) to return its output to our agent!

In [15]:
uncompiled_graph.add_edge("action", "agent")

Let's look at the final visualization.

![image](https://i.imgur.com/NWO7usO.png)

All that's left to do now is to compile our workflow - and we're off!

In [16]:
simple_agent_graph = uncompiled_graph.compile()

#### ❓ Question #2:

1. Is there any specific limit to how many times we can cycle?

2. If not, how could we impose a limit to the number of cycles?

-------------------------------------------------------------------------------------------------------------------------
##### ✅ Answer:

1. Based on the langgraph [documentation](https://langchain-ai.github.io/langgraph/concepts/low_level/#recursion-limit), the limit is set to 25 cycles (recursion limit).
2. The same documentation states that this limit can also be set by using the .invoke method. Example:
```python
    graph.invoke(inputs, config={"recursion_limit": 5}, context={"llm": "anthropic"})
```
-------------------------------------------------------------------------------------------------------------------------

## Using Our Graph

Now that we've created and compiled our graph - we can call it *just as we'd call any other* `Runnable`!

Let's try out a few examples to see how it fairs:

In [27]:
# What's a Runnable?
# A Runnable is LangChain's core abstraction for anything that can be executed/run.
# Simple definition: If you can call .invoke(), .stream(), or .astream() on it, it's a Runnable.

from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="How are technical professionals using AI to improve their work?")]}

async for chunk in simple_agent_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='Technical professionals are using AI in various ways to enhance their work, including automating repetitive tasks, improving decision-making, analyzing large datasets, developing new products and services, and optimizing processes. They leverage AI for tasks such as machine learning model development, natural language processing, computer vision, predictive analytics, and automation. This integration helps increase efficiency, accuracy, and innovation across different industries. Would you like specific examples from particular fields or industries?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 163, 'total_tokens': 251, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14',

Let's look at what happened:

1. Our state object was populated with our request
2. The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. The conditional edge received the state object, found the "tool_calls" `additional_kwarg`, and sent the state object to the action node
4. The action node added the response from the OpenAI function calling endpoint to the state object and passed it along the edge to the agent node
5. The agent node added a response to the state object and passed it along the conditional edge
6. The conditional edge received the state object, could not find the "tool_calls" `additional_kwarg` and passed the state object to END where we see it output in the cell above!

Now let's look at an example that shows a multiple tool usage - all with the same flow!

In [30]:
inputs = {"messages" : [HumanMessage(content="Search Arxiv for the A Comprehensive Survey of Deep Research paper, then search each of the authors to find out where they work now using Tavily!")]}

async for chunk in simple_agent_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node == "action":
          print(f"Tool Used: {values['messages'][0].name}")
        print(values["messages"])

        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LqoqJVhEPMfzwCZ1UecLl2n5', 'function': {'arguments': '{"query": "A Comprehensive Survey of Deep Research"}', 'name': 'arxiv'}, 'type': 'function'}, {'id': 'call_l8Vvnu7lw1FOOA3medDbRX1k', 'function': {'arguments': '{"query": "A Comprehensive Survey of Deep Research paper"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 182, 'total_tokens': 241, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'id': 'chatcmpl-CLL8A2neSH9Va1jDMjDAbRqGcXmck', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--263a9c06-904e-43

#### 🏗️ Activity #2:

Please write out the steps the agent took to arrive at the correct answer.

--------------------------------------------------------------------------------------------------------------------------------
##### ✅ Answer:

This interaction has 2 total cycles:

1. The user sends its initial question, the state only contains **[HumanMessage]**

Cicle #1

2. Agent node reads the question and decides what tools to use. It adds to state: **AIMessage(tool_calls: [arxiv, tavily_search])**
3. The conditional edge checks whether there are tool_calls, in this case there are so → go to action node
4. Action node executes both tools (arxiv + tavily) and gets paper info, and adds to state **ToolMessage(arxiv results), ToolMessage(tavily results)**
5. Action node automatically routes back to agent node

Cicle #2

6. Agent node reads all results, sees author names, decides to search each author. It adds to state **AIMessage(tool_calls: [tavily(Renjun Xu), tavily(Jingwen Peng)])**
7. (AGAIN) The conditional edge checks whether there are tool_calls, in this case there are so → go to action node
8. Action node executes both author searches and gets their current jobs. Adds to state **ToolMessage(Renjun results), ToolMessage(Jingwen results)**
9. Action node automatically routes back to agent node

Final Phase:

10. Agent node reads everything, has all the info needed, writes final answer. Adds to state **AIMessage("Renjun Xu is at... Jingwen Peng is at...")** and NO TOOL CALL
11. The conditional edge checks whether there are tool_calls, in this case there are so → go to END

So as we can see, the graph cycled twice - once to get the paper, once to get author info - before finishing.

--------------------------------------------------------------------------------------------------------------------------------

# 🤝 Breakout Room #2

## Part 1: LangSmith Evaluator

### Pre-processing for LangSmith

To do a little bit more preprocessing, let's wrap our LangGraph agent in a simple chain.

In [32]:
# Converts simple text input into the state format your graph expects
def convert_inputs(input_object):
  return {"messages" : [HumanMessage(content=input_object["text"])]}

# Extracts just the final answer from the full state
def parse_output(input_state):
  return {"answer" : input_state["messages"][-1].content}

# Chaining runnables together with "|" 
agent_chain_with_formatting = convert_inputs | simple_agent_graph | parse_output

#Input → convert_inputs → simple_agent_graph → parse_output → Output

# Runnable
agent_chain_with_formatting.invoke({"text" : "What is Deep Research?"})

{'answer': 'Deep Research typically refers to an in-depth and comprehensive investigation or analysis into a specific topic, subject, or field. It involves gathering detailed information, examining various sources, and analyzing data thoroughly to gain a profound understanding. Deep Research is often used in academic, scientific, technological, and business contexts to develop insights, inform decision-making, or advance knowledge.\n\nIf you are referring to a specific organization, product, or platform named "Deep Research," please provide more context so I can give a more precise answer.'}


```yaml
# You invoke with simple format
agent_chain_with_formatting.invoke({"text": "What is Deep Research?"})

# Step 1: convert_inputs transforms it
{"text": "What is Deep Research?"} 
  → {"messages": [HumanMessage("What is Deep Research?")]}

# Step 2: simple_agent_graph processes it (runs your whole agent)
{"messages": [HumanMessage("What is Deep Research?")]}
  → {"messages": [HumanMessage(...), AIMessage(...), ToolMessage(...), AIMessage("Deep Research is...")]}

# Step 3: parse_output extracts the answer
{"messages": [..., AIMessage("Deep Research is...")]}
  → {"answer": "Deep Research is..."}

# Final result
{"answer": "Deep Research is..."}
```

### Task 1: Creating An Evaluation Dataset

Just as we saw last week, we'll want to create a dataset to test our Agent's ability to answer questions.

In order to do this - we'll want to provide some questions and some answers. Let's look at how we can create such a dataset below.

```python
questions = [
    {
        "inputs" : {"text" : "Who were the main authors on the 'A Comprehensive Survey of Deep Research: Systems, Methodologies, and Applications' paper?"},
        "outputs" : {"must_mention" : ["Peng", "Xu"]}   
    },
    ...,
    {
        "inputs" : {"text" : "Where do the authors of the 'A Comprehensive Survey of Deep Research: Systems, Methodologies, and Applications' work now?"},
        "outputs" : {"must_mention" : ["Zhejiang", "Liberty Mutual"]}
    }
]
```

#### 🏗️ Activity #3:

Please create a dataset in the above format with at least 5 questions that pertain to the cohort use-case (more information [here](https://www.notion.so/Session-4-RAG-with-LangGraph-OSS-Local-Models-Eval-w-LangSmith-26acd547af3d80838d5beba464d7e701#26acd547af3d81d08809c9c82a462bdd)), or the use-case you're hoping to tackle in your Demo Day project.

In [34]:
questions = [
    {
        "inputs": {"text": "What were the main topics of ChatGPT conversations according to this research?"},
        "outputs": {"must_mention": ["practical guidance", "seeking information", "writing"]}
    },
    {
        "inputs": {"text": "How did work-related versus non-work-related ChatGPT usage change over time?"},
        "outputs": {"must_mention": ["non-work increased", "53% to 73%", "work-related declined"]}
    },
    {
        "inputs": {"text": "What percentage of ChatGPT messages were related to computer programming?"},
        "outputs": {"must_mention": ["4.2%", "relatively small"]}
    },
    {
        "inputs": {"text": "How did the researchers classify user intent in ChatGPT messages?"},
        "outputs": {"must_mention": ["asking", "doing", "expressing"]}
    },
    {
        "inputs": {"text": "What were the most common work activities associated with ChatGPT usage according to O*NET classification?"},
        "outputs": {"must_mention": ["getting information", "documenting information", "making decisions", "solving problems"]}
    },
    {
        "inputs": {"text": "How did gender gaps in ChatGPT usage evolve from launch to 2025?"},
        "outputs": {"must_mention": ["80% masculine", "narrowed", "parity", "closed"]}
    },
    {
        "inputs": {"text": "What privacy measures did the researchers implement when analyzing ChatGPT messages?"},
        "outputs": {"must_mention": ["automated classifiers", "no human viewing", "PII scrubbed", "data clean room"]}
    }
]

Now we can add our dataset to our LangSmith project using the following code which we saw last Thursday!

In [37]:
from langsmith import Client

# Connects to LangSmith (a monitoring/evaluation platform for LLM apps)
client = Client()

# Generate a Unique Dataset Name, Creates a unique name like: "Simple Search Agent - Evaluation Dataset - a3b2c1d4"
dataset_name = f"Simple Search Agent - Evaluation Dataset - {uuid4().hex[0:8]}"

# Creates an empty dataset container in LangSmith to hold your test questions
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions about the cohort use-case to evaluate the Simple Search Agent."
)

# Populates the dataset with actual test questions
client.create_examples(
    dataset_id=dataset.id,
    examples=questions
)

{'example_ids': ['83637ebf-7fa6-46c5-9a79-845215459039',
  '5d601100-6fcd-4bad-b874-e2505974ef24',
  '42e82f15-bb73-45b0-92d8-66e72145aedf',
  'a18386ab-29ad-49c8-896c-0b34a4e32adc',
  'c146f3d1-5116-4b4d-8aa6-6ee0e919c295',
  '801df840-045c-415a-b327-d57d9a7f0742',
  'd2ab5de6-b1b8-4eb3-9f86-ca6038a299ab'],
 'count': 7}

### Task 2: Adding Evaluators

Let's use the OpenEvals library to product an evaluator that we can then pass into LangSmith!

> NOTE: Examine the `CORRECTNESS_PROMPT` below!

In [39]:
# This code is importing and displaying a pre-written evaluation prompt from the openevals library

from openevals.prompts import CORRECTNESS_PROMPT
print(CORRECTNESS_PROMPT)

# WHY USE THIS?
# When evaluating your agent, you need something to judge if answers are good. This prompt template:

# Standardizes evaluation - Same criteria for all test cases
# Uses an LLM as a judge - Another AI evaluates your agent's answers
# Saves time - No need to write your own evaluation prompt

# THE FLOW
# 1. Your agent answers: "Deep Research is an AI system..."
# 2. Evaluator uses CORRECTNESS_PROMPT to ask another LLM:
#    "Is this answer correct?"
# 3. Judge LLM responds: "CORRECT" or "INCORRECT"
# 4. Result gets logged in LangSmith

You are an expert data labeler evaluating model outputs for correctness. Your task is to assign a score based on the following rubric:

<Rubric>
  A correct answer:
  - Provides accurate and complete information
  - Contains no factual errors
  - Addresses all parts of the question
  - Is logically consistent
  - Uses precise and accurate terminology

  When scoring, you should penalize:
  - Factual errors or inaccuracies
  - Incomplete or partial answers
  - Misleading or ambiguous statements
  - Incorrect terminology
  - Logical inconsistencies
  - Missing key information
</Rubric>

<Instructions>
  - Carefully read the input and output
  - Check for factual accuracy and completeness
  - Focus on correctness of information rather than style or verbosity
</Instructions>

<Reminder>
  The goal is to evaluate factual correctness and completeness of the response.
</Reminder>

<input>
{inputs}
</input>

<output>
{outputs}
</output>

Use the reference outputs below to help you evaluate the

In [40]:
from openevals.llm import create_llm_as_judge

correctness_evaluator = create_llm_as_judge(
    prompt=CORRECTNESS_PROMPT,           # The evaluation instructions
    model="openai:o3-mini",              # The LLM that will evaluate your agent's answers ---- very impactful to the final score
    feedback_key="correctness",          # Name for this metric
    )

Let's also create a custom Evaluator for our created dataset above - we do this by first making a simple Python function!

In [43]:
def must_mention(inputs: dict, outputs: dict, reference_outputs: dict) -> float:
  # determine if the phrases in the reference_outputs are in the outputs
  # Get the list of required phrases
  required = reference_outputs.get("must_mention") or []
  # Check if ALL required phrases are in the answer
  score = all(phrase in outputs["answer"] for phrase in required)
  # Return 1.0 (pass) or 0.0 (fail)
  return score

# Example
# Test case in your dataset:
# {
#     "text": "Who are the authors of the Deep Research paper?",
#     "must_mention": ["Renjun Xu", "Jingwen Peng"]
# }

# Agent's answer:
# "The authors are Renjun Xu and Jingwen Peng from Zhejiang University."

# Evaluation:
# Check: Is "Renjun Xu" in the answer? ✅ Yes
# Check: Is "Jingwen Peng" in the answer? ✅ Yes
# Score: 1.0 (Pass)

# If agent answered:
# "The paper was written by researchers at Zhejiang University."
# Evaluation:

# Check: Is "Renjun Xu" in the answer? ❌ No
# Score: 0.0 (Fail)


#### ❓ Question #4:

What are some ways you could improve this metric as-is?

> NOTE: Alternatively you can suggest where gaps exist in this method.

-----------------------------------------------------------------------------------------------------------------------------------------------------------
##### ✅ Answer:

The **must_mention** metric has two main problems. First, it uses exact text matching, which is too strict - it fails if the agent paraphrases correctly (saying "global warming" instead of "climate change") or makes small spelling variations. Second, it gives no partial credit, so an answer with 4 out of 5 required phrases gets the same zero score as an answer with none, even though one is clearly better.

To improve it, add fuzzy matching or semantic similarity so the metric recognizes when concepts are present even if worded differently. Also implement partial scoring - if 80% of required phrases are mentioned, give a 0.8 score instead of 0. This makes the metric more realistic about how well the agent actually performed.

-----------------------------------------------------------------------------------------------------------------------------------------------------------

Task 3: Evaluating

All that is left to do is evaluate our agent's response!

In [44]:
results = client.evaluate(
    agent_chain_with_formatting,
    data=dataset.name,
    evaluators=[correctness_evaluator, must_mention],
    experiment_prefix="simple_agent, baseline",  # optional, experiment name prefix
    description="Testing the baseline system.",  # optional, experiment description
    max_concurrency=4, # optional, add concurrency
)

View the evaluation results for experiment: 'simple_agent, baseline-546c90f7' at:
https://smith.langchain.com/o/bb67da1a-f981-488f-9c09-ce7a028c911d/datasets/3b6f7c63-18b4-47da-8bcf-5fa6897dd6bd/compare?selectedSessions=1b5a16b2-d72c-44ba-bfa5-bb305df7bedf




0it [00:00, ?it/s]

## Part 2: LangGraph with Helpfulness:

### Task 3: Adding Helpfulness Check and "Loop" Limits

Now that we've done evaluation - let's see if we can add an extra step where we review the content we've generated to confirm if it fully answers the user's query!

We're going to make a few key adjustments to account for this:

1. We're going to add an artificial limit on how many "loops" the agent can go through - this will help us to avoid the potential situation where we never exit the loop.
2. We'll add to our existing conditional edge to obtain the behaviour we desire.

First, let's define our state again - we can check the length of the state object, so we don't need additional state for this.

In [45]:
class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

Now we can set our graph up! This process will be almost entirely the same - with the inclusion of one additional node/conditional edge!

#### 🏗️ Activity #4:

Please write markdown for the following cells to explain what each is doing.

##### YOUR MARKDOWN HERE

Create a new StateGraph with AgentState and add two nodes:
- **agent node**: Calls the LLM to decide what action to take
- **action node**: Executes the tools selected by the agent

In [47]:
graph_with_helpfulness_check = StateGraph(AgentState)

graph_with_helpfulness_check.add_node("agent", call_model)
graph_with_helpfulness_check.add_node("action", tool_node)

##### YOUR MARKDOWN HERE

Configure the graph to start at the "agent" node when execution begins. (Set "agent" node as the Entrypoint)


In [49]:
graph_with_helpfulness_check.set_entry_point("agent")

##### YOUR MARKDOWN HERE

Create a function that determines where to route after the agent responds. It checks three conditions in order:

1. **Tool calls present?** → Route to "action" node to execute tools
2. **More than 10 messages?** → End execution (prevents infinite loops)
3. **Response helpful enough?** → Use GPT-4.1-mini to evaluate if the answer is helpful:
   - If helpful (contains 'Y') → End execution
   - If not helpful → Route back to "agent" to try again

This adds a quality check loop where unhelpful responses trigger another attempt.

In [50]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def tool_call_or_helpful(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  initial_query = state["messages"][0]
  final_response = state["messages"][-1]

  if len(state["messages"]) > 10:
    return "END"

  prompt_template = """\
  Given an initial query and a final response, determine if the final response is extremely helpful or not. Please indicate helpfulness with a 'Y' and unhelpfulness as an 'N'.

  Initial Query:
  {initial_query}

  Final Response:
  {final_response}"""

  helpfullness_prompt_template = PromptTemplate.from_template(prompt_template)

  helpfulness_check_model = ChatOpenAI(model="gpt-4.1-mini")

  helpfulness_chain = helpfullness_prompt_template | helpfulness_check_model | StrOutputParser()

  helpfulness_response = helpfulness_chain.invoke({"initial_query" : initial_query.content, "final_response" : final_response.content})

  if "Y" in helpfulness_response:
    return "end"
  else:
    return "continue"

##### YOUR MARKDOWN HERE

Add a conditional edge from the "agent" node that uses the `tool_call_or_helpful` function to decide routing. The routing map specifies:
- "continue" → Loop back to "agent" node (answer wasn't helpful enough)
- "action" → Go to "action" node (tools needed)
- "end" → Stop execution (answer is helpful or limit reached)

In [51]:
graph_with_helpfulness_check.add_conditional_edges(
    "agent",
    tool_call_or_helpful,
    {
        "continue" : "agent",
        "action" : "action",
        "end" : END
    }
)

##### YOUR MARKDOWN HERE

This adds a regular edge from "action" back to "agent" - meaning after tools run, always return to the agent.

In [52]:
graph_with_helpfulness_check.add_edge("action", "agent")

##### YOUR MARKDOWN HERE

Compile the graph into an executable runnable. This finalizes the structure and makes it ready to process requests.


In [53]:
agent_with_helpfulness_check = graph_with_helpfulness_check.compile()

##### YOUR MARKDOWN HERE

Run the agent with a test question about Deep Research Agents. Stream updates as each node executes, printing which node is active and what messages were added to the state.Retry

In [54]:
inputs = {"messages" : [HumanMessage(content="What are Deep Research Agents?")]}

async for chunk in agent_with_helpfulness_check.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='Deep Research Agents are advanced AI systems designed to assist with in-depth research tasks. They leverage deep learning techniques and large datasets to analyze complex information, generate insights, and support decision-making across various fields such as science, technology, medicine, and more. These agents can automate literature reviews, extract relevant data from vast sources, and provide comprehensive summaries, making research more efficient and thorough. Would you like me to find more detailed or specific information about Deep Research Agents?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 158, 'total_tokens': 251, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-

## Part 3: LangGraph for the "Patterns" of GenAI

### Task 4: Helpfulness Check of Gen AI Pattern Descriptions

Let's ask our system about the 3 main patterns in Generative AI:

1. Context Engineering
2. Fine-tuning
3. Agents

In [55]:
patterns = ["Context Engineering", "Fine-tuning", "LLM-based agents"]

In [56]:
for pattern in patterns:
  what_is_string = f"What is {pattern} and when did it break onto the scene??"
  inputs = {"messages" : [HumanMessage(content=what_is_string)]}
  messages = agent_with_helpfulness_check.invoke(inputs)
  print(messages["messages"][-1].content)
  print("\n\n")

Context Engineering is a relatively new interdisciplinary field that focuses on designing, managing, and utilizing contextual information to improve the functionality and adaptability of systems, particularly in areas like artificial intelligence, human-computer interaction, and pervasive computing. It involves understanding and engineering the context in which systems operate to enhance their performance, relevance, and user experience.

The concept of Context Engineering began gaining attention in the early 2000s with the rise of ubiquitous computing and context-aware systems. It became more prominent as researchers and industry professionals recognized the importance of context in creating intelligent, responsive systems that can adapt to dynamic environments and user needs.

Would you like me to find more detailed or specific information about the origins and development of Context Engineering?



Fine-tuning is a machine learning technique used to adapt a pre-trained model to a sp